# Solución 1

## Detección de intrusos en redes

Estos datos fueron usados para la edición de 1999 del KDD cup. Los datos fueron generados por Lincoln Labs: _Nueve semanas de registro de paquetes TCP fueron recolectadas para una red LAN de una oficina de las fuerzas aéreas de USA._ Durante el uso de la LAN, _varios ataques_ fueron ejecutados por el personal. El paquete crudo fue agregado junto con la información de la conexión. 

Para cada registro, algunas características extra fueron derivadas, basados en conocimiento del dominio sobre ataques a redes; _hay 38 tipos diferentes de ataques, pertenecientes a 4 categorías principales_. Algunos tipos de ataque aparecen solo en los datos de prueba(test data), y las frecuencias de los tipo de ataque en los conjuntos de entrenamiento y prueba no son las mismas(para hacerlo más realista). Información adicional sobre los datos puede ser encontrada en (http://kdd.ics.uci.edu/databases/kddcup99/task.html) y los resumenes de los resultados de la competencia KDD cup (http://cseweb.ucsd.edu/~elkan/clresults.html). En la última página también se indica que hay una matriz de costo asociada con las equivocaciones.  El ganador de la competencia usó árboles de decisión C5 en combinación con boosting y bagging.

**Referencias**:
- PNrule: _A New Framework for Learning Classifier Models in Data Mining (A Case-Study in Network Intrusion Detection) (2000) by R. Agarwal and M. V. Joshi_. This paper proposes a new, very simple rule learning algorithm, and tests it on the network intrusion dataset. In the first stage, rules are learned to identify the target class, and then in the second stage, rules are learned to identify cases that were incorrectly classified as positive according to the first rules.

## Pasos a llevar a cabo en la solución 1

- Cargar las librerías a utilizar
- Cargar los datos e importarlos a un dataframe
- Visualizar los datos
- Limpiar y transformar los datos
- Códificar los datos
- Seleccionar los parámetros más importantes
- Separando el conjunto de datos de entrenamiento y de validación
- Selección de algoritmos y métodos
- Validación Cruzada con KFold
- Subir los resultados al Github

## Cargar las librerías a utilizar

In [ ]:
%matplotlib inline

from time import time
from itertools import product
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import graphviz
from sklearn import metrics
from sklearn.metrics import make_scorer, accuracy_score, confusion_matrix
from sklearn.svm import SVC
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from IPython.display import display, HTML

from egironML import EDA # Modulo personal para hacer mas claro el ejercicio

plt.style.use('seaborn-white')

## Cargar los datos e importarlos a un dataframe

In [ ]:
# Cargamos los datos de Detección de intrusos por medio de un modelo personalizado
ataques_10perc, ataques_correg_test_10perc = EDA.load_attack_rawData()

## Visualizar los datos

In [ ]:
ataques_10perc.head()

In [ ]:
EDA.printall(ataques_correg_test_10perc, 15)

In [ ]:
# ataques_correg_test_10perc.sample(3)

In [ ]:
# Veamos la distribucción de los ataques
ataques_10perc.attack_types.value_counts().plot(kind='bar', grid=True, figsize=(15, 6))

In [ ]:
ataques_10perc.attack_types.value_counts()

In [ ]:
ataques_correg_test_10perc.attack_types.value_counts().plot(kind='bar', grid=True, figsize=(15, 6))

In [ ]:
ataques_correg_test_10perc.attack_types.value_counts()

In [ ]:
print("Cantidad de ataques (Entrenamiento): ",len(ataques_10perc.attack_types.unique()))
print("Cantidad de ataques (Validación): ",len(ataques_correg_test_10perc.attack_types.unique()))

## Limpiar y transformar los datos

Como se aprecia en los datos anteriores, la cantidad de ataques son diferentes en en el conjunto de datos de entrenamiento y valdación. Igualmente poseen un punto al final de cada atributo de tipo de ataque. Por ultimo hay algunas variables o predictores de tipo categórico.

In [ ]:
# Reemplazamos el . en los vlores del precitor de tipos de ataques.
EDA.replace_column_string(ataques_10perc, 'attack_types', '.', '')
EDA.replace_column_string(ataques_correg_test_10perc, 'attack_types', '.', '')

In [ ]:
ataques_10perc.attack_types.unique()

In [ ]:
ataques_correg_test_10perc.attack_types.unique()

In [ ]:
# Verificamos el tipo de dato en cada atributo o predictor
ataques_10perc.info()
ataques_correg_test_10perc.info()

Como se puede apreciar el conjunto de entrenamiento tiene **494021** observaciones y el de validación **311029**; todas las vairables estan completas por lo que no hay valores faltantes o nulos. De igual forma se notan 4 variables categóricas (*protocol_type, service, flag y attack_types*). 

In [ ]:
# Varificamos de nuevo o de otra forma si hay datos faltantes o nulos
# ataques_10perc.isnull().sum()
EDA.hasNull(ataques_10perc)

In [ ]:
ataques_correg_test_10perc.apply(EDA.hasNull)

In [ ]:
# Eliminar duplicados
EDA.delduplicates(ataques_10perc)
ataques_correg_test_10percAll = ataques_correg_test_10perc # Salvamos una copia con todos los datos para validar
EDA.delduplicates(ataques_correg_test_10perc)

Es necesario tener en cuenta que los resultados que se muestran en las matrices de confusión del ganador del KDD Cup son con todos los 311029 datos incluyendo los duplicados.

In [ ]:
def create_category_attack(df):
        # Crear un nuevo predictor de categorías de tipos de ataques
        df['attack_category'] = df.attack_types.map({'normal': 'normal', 'buffer_overflow':'u2r', 
                                                       'loadmodule':'u2r', 'perl':'u2r', 'neptune':'dos',
                                                       'smurf':'dos','guess_passwd':'r2l', 'pod':'dos', 
                                                       'teardrop':'dos', 'portsweep':'probe','ipsweep':'probe',
                                                       'land':'dos','ftp_write':'r2l','back':'dos','imap':'r2l',
                                                       'satan':'probe','phf':'r2l','nmap':'probe',
                                                       'multihop':'r2l','warezmaster':'r2l','warezclient':'r2l',
                                                       'spy':'r2l','rootkit':'u2r', 'snmpgetattack':'unknown', 
                                                       'named':'unknown', 'xlock':'unknown', 'xsnoop':'unknown', 
                                                        'sendmail':'unknown', 'saint':'unknown','apache2':'unknown', 
                                                        'udpstorm':'unknown','xterm':'unknown', 'mscan':'unknown', 
                                                        'processtable':'unknown', 'ps':'unknown','httptunnel':'unknown', 
                                                        'worm':'unknown', 'mailbomb':'unknown','sqlattack':'unknown', 
                                                        'snmpguess':'unknown' })

        print("Categorías encontradas:", df.attack_category.unique())

In [ ]:
# Ahora transformamos algunos datos como lo son los tipo de ataques
# Agrupamos los tipos de ataques en las 4 categorías recomendadas
EDA.create_category_attack(ataques_10perc)
EDA.create_category_attack(ataques_correg_test_10perc)
EDA.create_category_attack(ataques_correg_test_10percAll)

In [ ]:
# Ahora creamos una variable binaria (0/1 o No/Yes o good/bad) para guardar las conexiones con o sin ataques
# Esto nos permite aplicar modelos de regresión mas adelante
EDA.create_category_binAttack(ataques_10perc)
EDA.create_category_binAttack(ataques_correg_test_10perc)
EDA.create_category_binAttack(ataques_correg_test_10percAll)

In [ ]:
gdata = ataques_10perc.attack_category.value_counts()
sns.barplot(x=gdata.keys().tolist(), y=gdata.data.tolist())

In [ ]:
gdata = ataques_correg_test_10perc.attack_category.value_counts()
sns.barplot(x=gdata.keys().tolist(), y=gdata.data.tolist())

In [ ]:
import plotly
from plotly.graph_objs import Bar, Scatter, Layout, Figure
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
plotly.offline.init_notebook_mode(connected=True)

gdata = ataques_correg_test_10perc.attack_types.value_counts()
attackctg = Bar(x=gdata.keys().tolist(), y=gdata.data.tolist(), name='Attack', marker=dict(color='#ff1123') )

data = [attackctg]
layout = Layout(title="Attack Types", xaxis=dict(title='Type of Attack'), 
                yaxis=dict(title='Count of Attacks (in thousands)'))
fig = Figure(data=data, layout=layout)

iplot(fig)

In [ ]:
corr = ataques_correg_test_10perc.corr()
f, ax = plt.subplots(figsize=(12, 12))
sns.heatmap(corr, mask=np.zeros_like(corr, dtype=np.bool), cmap=sns.diverging_palette(220, 10, as_cmap=True),
            square=False, ax=ax)

In [ ]:
ataques_correg_test_10perc.columns

In [ ]:
datatograph = ataques_correg_test_10perc[['duration', 'src_bytes',
       'dst_bytes', 'land', 'wrong_fragment', 'urgent', 'hot',
       'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell',
       'su_attempted', 'num_root', 'num_file_creations', 'num_shells',
       'num_access_files', 'num_outbound_cmds', 'is_host_login',
       'is_guest_login', 'count', 'srv_count', 'serror_rate',
       'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate',
       'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count',
       'dst_host_srv_count', 'dst_host_same_srv_rate',
       'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate',
       'dst_host_srv_diff_host_rate', 'dst_host_serror_rate',
       'dst_host_srv_serror_rate', 'dst_host_rerror_rate',
       'dst_host_srv_rerror_rate',
       'attack_category_bin']]
pd.plotting.scatter_matrix(datatograph, alpha = 0.3, figsize = (14,14), diagonal = 'kde')

/anaconda/envs/gisenv/lib/python3.6/site-packages/matplotlib/axes/_base.py:2917: UserWarning:

Attempting to set identical left==right results
in singular transformations; automatically expanding.
left=0.0, right=0.0

/anaconda/envs/gisenv/lib/python3.6/site-packages/matplotlib/axes/_base.py:3193: UserWarning:

Attempting to set identical bottom==top results
in singular transformations; automatically expanding.
bottom=0.0, top=0.0



LinAlgError: singular matrix

## Códificar los datos

In [31]:
def create_dummies(categorical_variables):
        for variable in categorical_variables:
            # Crear variables dummy 
            dummies = pd.get_dummies(X[variable], prefix=variable).iloc[:, 1:]
            # Update X para incluir las dummies y eliminar la variable principal
            X = pd.concat([X, dummies], axis=1)
            X.drop([variable], axis=1, inplace=True) # Eliminar las variables categóricas

## Seleccionar los parámetros más importantes

## Separando el conjunto de datos de entrenamiento y de validación

## Selección de algoritmos y métodos

## Validación Cruzada con KFold

## Subir los resultados al Github